<a href="https://colab.research.google.com/github/DomizianoScarcelli/big-data-project/blob/item-based-cf/item_based_CF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Configuration

In [1]:
import os
def is_running_on_colab():
    return "COLAB_GPU" in os.environ

LOCAL = not is_running_on_colab()

In [2]:
#@title Download necessary libraries
if not LOCAL:
    !pip install pyspark -qq
    !pip install -U -q PyDrive -qq
    !apt install openjdk-8-jdk-headless -qq

In [18]:
#@title Imports
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import plotly

import pyspark
import pyspark.sql.functions as F
from pyspark.sql import SparkSession, DataFrame, Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ArrayType, FloatType, LongType
from pyspark import SparkContext, SparkConf
from pyspark.ml.linalg import SparseVector, DenseVector, VectorUDT

from tqdm.notebook import tqdm
import time
import gc

if not LOCAL:
    from google.colab import drive

from typing import Tuple, Dict, List
from functools import reduce
import pickle

In [4]:
#@title Set up variables
if not LOCAL:
    JAVA_HOME = "/usr/lib/jvm/java-8-openjdk-amd64"
    GDRIVE_DIR = "/content/drive"
    GDRIVE_HOME_DIR = GDRIVE_DIR + "/MyDrive"
    GDRIVE_DATA_DIR = GDRIVE_HOME_DIR + "/Big Data/datasets"
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_friendly_spotify_playlist_dataset")
    AUDIO_FEATURES_FILE = os.path.join(GDRIVE_DATA_DIR, "pyspark_track_features")
    LITTLE_SLICE_FILE = os.path.join(GDRIVE_DATA_DIR, "little_slice")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    LITTLE_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "little_slice_audio_features")
    MICRO_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "micro_slice_audio_features")
    SPLITTED_SLICE_AUDIO_FEATURES = os.path.join(GDRIVE_DATA_DIR, "splitted_pyspark_track_features")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
else:
    GDRIVE_DATA_DIR = os.path.abspath("./data")
    SAVED_DFS_PATH = os.path.join(GDRIVE_DATA_DIR, "saved_dfs")
    SAVED_MODELS = os.path.join(GDRIVE_DATA_DIR, "saved_models")
    DATASET_FILE = os.path.join(GDRIVE_DATA_DIR, "full_dataset")
    SMALL_SLICE_FLIE = os.path.join(GDRIVE_DATA_DIR, "small_slice")
    JAVA_HOME = "/opt/homebrew/opt/openjdk"
RANDOM_SEED = 42 # for reproducibility
os.environ["JAVA_HOME"] = JAVA_HOME
os.environ["PYSPARK_PYTHON"]="python"

In [5]:
#@title Create the session
conf = SparkConf().\
                set('spark.ui.port', "4050").\
                set('spark.executor.memory', '12G').\
                set('spark.driver.memory', '12G').\
                set('spark.driver.maxResultSize', '100G').\
                set("spark.executor.extraJavaOptions", "-XX:+UseG1GC").\
                setAppName("PySparkTutorial").\
                setMaster("local[*]")

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

23/06/22 19:52:57 WARN Utils: Your hostname, MacBook-Air-di-Domiziano.local resolves to a loopback address: 127.0.0.1; using 192.168.1.175 instead (on interface en0)
23/06/22 19:52:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/22 19:52:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/06/22 19:52:58 WARN Utils: Service 'SparkUI' could not bind on port 4050. Attempting port 4051.


In [6]:
if not LOCAL:
    drive.mount(GDRIVE_DIR, force_remount=True)

In [7]:
#@title Check if everything is ok
spark, sc._conf.getAll()


(<pyspark.sql.session.SparkSession at 0x161835480>,
 [('spark.executor.extraJavaOptions',
   '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.security.jgss/sun.security.krb5=ALL-UNNAMED -Djdk.reflect.useDirectMethodHandle=false -XX:+UseG1GC'),
  ('spark.driver.port', '59874'),
  ('spark.app.startTime', '1687456378005'),
  ('spa

# Data acquisition

In [8]:
song_schema = StructType([
    StructField("pos", IntegerType(), True),
    StructField("artist_name", StringType(), True),
    StructField("track_uri", StringType(), True),
    StructField("artist_uri", StringType(), True),
    StructField("track_name", StringType(), True),
    StructField("album_uri", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("album_name", StringType(), True)
])

playlist_schema = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", ArrayType(song_schema), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

playlist_schema_mapped = StructType([
    StructField("name", StringType(), True),
    StructField("collaborative", StringType(), True),
    StructField("pid", IntegerType(), True),
    StructField("modified_at", IntegerType(), True),
    StructField("num_tracks", IntegerType(), True),
    StructField("num_albums", IntegerType(), True),
    StructField("num_followers", IntegerType(), True),
    StructField("tracks", VectorUDT(), True),
    StructField("num_edits", IntegerType(), True),
    StructField("duration_ms", IntegerType(), True),
    StructField("num_artists", IntegerType(), True),
])

audio_features_schema = StructType([
    StructField("danceability", FloatType(), True),
    StructField("energy", FloatType(), True),
    StructField("key", IntegerType(), True),
    StructField("loudness", FloatType(), True),
    StructField("mode", IntegerType(), True),
    StructField("speechiness", FloatType(), True),
    StructField("acousticness", FloatType(), True),
    StructField("instrumentalness", FloatType(), True),
    StructField("liveness", FloatType(), True),
    StructField("valence", FloatType(), True),
    StructField("tempo", FloatType(), True),
    StructField("type", StringType(), True),
    StructField("id", StringType(), True),
    StructField("uri", StringType(), True),
    StructField("track_href", StringType(), True),
    StructField("analysis_url", StringType(), True),
    StructField("duration_ms", LongType(), True),
    StructField("time_signature", IntegerType(), True)
])




In [9]:
# playlist_df = spark.read.schema(playlist_schema).json(DATASET_FILE, multiLine=True)
slice_df = spark.read.schema(playlist_schema).json(SMALL_SLICE_FLIE, multiLine=True)
# slice_df = spark.read.schema(playlist_schema).json(LITTLE_SLICE_FILE, multiLine=True)
# audio_df = spark.read.schema(audio_features_schema).json(SPLITTED_SLICE_AUDIO_FEATURES, multiLine=True) #has less songs than expected

23/06/22 19:53:15 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


# Item-Based Collaborative Filtering

Item-Based Collaboartive Filtering is the "transpose" approach to user-based CF. This time we won't consider the users' feature vectors, but the items'.
An item's rating vector $\mathbf{r}_i$ is the vector of ratings given to the item $i$ for all the users.

Let $m$ be the number of users, $n$ the number of playlists, then $\mathbf{r}_i \in \mathbb{R}^m$ and $\mathbf{R} \in \mathbb{R}^{m \times n}$.

In order to make a prediction, we take the set of items $I_u$ rated by the user $u$, and we compute the set $I^k_u$ of the top-$k$ most similar items to $i$ rated by $u$, for each item $i \in I_u$. Once done that, we average the $k$ rating vectors weighting them by their respective similarity.

In [10]:
DEBUG = True

In [12]:
NUM_PLAYLISTS = 100_000
SONGS_EMBEDDINGS_PATH = os.path.join(SAVED_DFS_PATH, f"songs_embeddings-train-{NUM_PLAYLISTS}.json")
SONGS_INFO_DF = os.path.join(SAVED_DFS_PATH, f"songs_info_df-train-{NUM_PLAYLISTS}.json") #TODO: Little bug this is songs_df, meaning it hasn't got any info, but we don't actually care.
RATING_VECTOR_LENGTH_PATH = os.path.join(SAVED_DFS_PATH, f"songs_vector_length-{NUM_PLAYLISTS}.txt")
with open(RATING_VECTOR_LENGTH_PATH, "r") as f:
  RATING_VECTOR_LENGTH = int(f.read())

songs_embeddings = spark.read.schema(playlist_schema_mapped).json(SONGS_EMBEDDINGS_PATH)
songs_df = spark.read.json(SONGS_INFO_DF)

playlist_map_schema = StructType([
    StructField("track_uri", StringType(), True),
    StructField("embedding", VectorUDT(), True)
])
PLAYLIST_MAP_PATH = os.path.join(SAVED_DFS_PATH, f"playlist_map-{NUM_PLAYLISTS}.json")
playlist_map = spark.read.schema(playlist_map_schema).json(PLAYLIST_MAP_PATH)

In [13]:
TRAIN_DF_PATH = os.path.join(SAVED_DFS_PATH, f"train_df-{NUM_PLAYLISTS}.json")
TEST_DF_PATH = os.path.join(SAVED_DFS_PATH, f"test_df-{NUM_PLAYLISTS}.json")

train_df = spark.read.schema(playlist_schema).json(TRAIN_DF_PATH)
test_df = spark.read.schema(playlist_schema).json(TEST_DF_PATH)

In [14]:
def jaccard_similarity(vector_1: SparseVector, vector_2: SparseVector) -> float:
  """
  Computes the Jaccard Similarity between two sparse binary vectors
  """
  # Convert SparseVectors to sets
  set1 = set(vector_1.indices)
  set2 = set(vector_2.indices)

  # Calculate the intersection and union of the sets
  intersection = len(set1.intersection(set2))
  union = len(set1.union(set2))

  # Calculate the similarity
  similarity = intersection / union

  return similarity

The similarity between each couple is intractable, we can cluster the similar tracks in buckets using Locally Sensitive Hashing.

In [15]:
from pyspark.ml.feature import MinHashLSH, MinHashLSHModel

NUM_HASH_TABLES = 10
LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"lsh_model-{NUM_HASH_TABLES}-{NUM_PLAYLISTS}.pickle")
if os.path.exists(LSH_MODEL_PATH):
  model = MinHashLSHModel.load(LSH_MODEL_PATH)
else:
  mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=NUM_HASH_TABLES)
  model = mh.fit(playlist_map)
  model.save(LSH_MODEL_PATH)

model.transform(playlist_map).show(truncate=False)

+------------------------------------+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|track_uri                           |embedding                                                                        |hashes                                                                                                                                                               |
+------------------------------------+---------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|spotify:track:001BVhvaZTf2icV88rU3DA|(100001,[7109,25065,41600,60248,61080,65885,70937],[1.0,1.0,1.0,1.0,1.0,1.0,1.0])|[[4.00933355E8], [4

Since the MinHasLSH model is not so fast, let's try with another type of LSH model

In [21]:
# from pyspark.ml.feature import BucketedRandomProjectionLSH, BucketedRandomProjectionLSHModel

# BRP_LSH_MODEL_PATH = os.path.join(SAVED_DFS_PATH, f"brp_lsh_model-{NUM_PLAYLISTS}.pickle")
# if os.path.exists(BRP_LSH_MODEL_PATH):
#   model = BucketedRandomProjectionLSHModel.load(BRP_LSH_MODEL_PATH)
# else:
#   brp = BucketedRandomProjectionLSH(inputCol="embedding", outputCol="hashes", numHashTables=5, bucketLength=1.0)
#   model = brp.fit(playlist_map)
#   model.save(BRP_LSH_MODEL_PATH)

# model.transform(playlist_map).show()

In [16]:
#TODO: Remove this once tested
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("distCol")).withColumnRenamed("distCol", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding", "hashes").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

if DEBUG:
  key = playlist_map.first()[1]
  neigh = model.approxNearestNeighbors(playlist_map, key, 10)
  merged_df_2 = merge_aggregate_df(transform_to_aggregate(neigh), neigh)
  merged_df_2.show() 

+--------------------+--------------------+--------------------+------------------+------------------+
|           track_uri|           embedding|              hashes|       squared_sum|               sum|
+--------------------+--------------------+--------------------+------------------+------------------+
|spotify:track:4OE...|(100001,[70937],[...|[[6.53736303E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:558...|(100001,[65885],[...|[[4.00933355E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:50r...|(100001,[70937],[...|[[6.53736303E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:2Eh...|(100001,[70937],[...|[[6.53736303E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:3lk...|(100001,[70937],[...|[[6.53736303E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:3Ff...|(100001,[65885],[...|[[4.00933355E8], ...|1.4693877551020411|1.7142857142857144|
|spotify:track:3De...|(100001,[70937],[...|[[6.53736303E8], ...|1.4693877

In [20]:
def df_to_dict(df: DataFrame) -> DataFrame:
  return df.select("track_uri", "distCol").rdd.collectAsMap()

def merge_dicts(d1: Dict[str, float], d2: Dict[str, float]) -> Dict[str, float]:
  for key, value in d2.items():
    if key in d1:
      if type(d1[key]) is float:
        d1[key] = [d1[key]]
      d1[key] += [value]
    else:
      d1[key] = [value]
  return d1

if DEBUG:
  key = playlist_map.first()[1]
  neigh = model.approxNearestNeighbors(playlist_map, key, 10).cache()
  merged_df_2 = merge_dicts(df_to_dict(neigh), df_to_dict(neigh))
  print(merged_df_2)

{'spotify:track:001BVhvaZTf2icV88rU3DA': [0.0, 0.0], 'spotify:track:1iO2inxYIzmPnMuDFfU1Rl': [0.8571428571428572, 0.8571428571428572], 'spotify:track:3Ff2kaO1uxXjd9HkHfMw4h': [0.8571428571428572, 0.8571428571428572], 'spotify:track:2EhgEpfn3U0lmpryqDujwt': [0.8571428571428572, 0.8571428571428572], 'spotify:track:3lkFKOOQRp1AqWk2PPAW6B': [0.8571428571428572, 0.8571428571428572], 'spotify:track:4OEHuq3q8kjkPS1jKI96JP': [0.8571428571428572, 0.8571428571428572], 'spotify:track:6kucDoXP5pBdFA7GxwgFP2': [0.8571428571428572, 0.8571428571428572], 'spotify:track:3DeMqzJj9477nCcyTXl3Ye': [0.8571428571428572, 0.8571428571428572], 'spotify:track:558Km9MuklF6yKJDVVjIli': [0.8571428571428572, 0.8571428571428572], 'spotify:track:50r5hQgwJ61tCwEL1maGsG': [0.8571428571428572, 0.8571428571428572]}


In [23]:
def extract_similar_songs(playlist_tracks: DataFrame, playlist_map, model, k=10, disable_pbar=False) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_map.join(F.broadcast(playlist_tracks), "track_uri").select("track_uri", "embedding")
  transformed_tracks_embeddings = model.transform(tracks_embedding).cache()
  k_neighs = []

  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors', disable=disable_pbar):
    k_neigh = model.approxNearestNeighbors(playlist_map, row["embedding"], k).cache()
    k_neighs.append(df_to_dict(k_neigh))

  aggregate_df = reduce(merge_dicts, k_neighs)

  transformed_tracks_embeddings.unpersist()
  return aggregate_df

if DEBUG:
  # first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*").distinct()
  first_playlist = train_df.filter("pid == 1005").select(F.explode("tracks")).select("col.*").distinct()
  recommendations = extract_similar_songs(first_playlist, playlist_map, model, k=10)

Extracting k-neighbors:   0%|          | 0/21 [00:00<?, ?it/s]

Exception in thread "serve RDD 112" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.ServerSocket.platformImplAccept(ServerSocket.java:655)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:631)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:588)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:546)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:65)
Exception in thread "serve RDD 138" java.net.SocketTimeoutException: Accept timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedAccept(NioSocketImpl.java:694)
	at java.base/sun.nio.ch.NioSocketImpl.accept(NioSocketImpl.java:738)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:690)
	at java.base/java.net.Serve

We are safe by using python dictionaries since the data will be very small, and so dictionaries will be faster than pyspark dataframes.

In [25]:
if DEBUG:
  import sys
  print(f"The reccomendation dictionary is {sys.getsizeof(recommendations) / 1_000} KB")

The reccomendation dictionary is 4.696 KB


In [26]:
def aggregate_recommendations(recommendations: Dict[str, float | List[float]]) -> DataFrame:
  aggregated = {}
  for key, value in recommendations.items():
    if type(value) is list:
      aggregated[key] = sum(x for x in value) / len(value)
    else:
      aggregated[key] = value
  # return {k: v for k, v in aggregated.items() if v != 0}
  # aggregated = sorted(aggregated.items(), key=lambda x: x[1])

  recommendations_schema = StructType([
      StructField("track_uri", StringType(), True),
      StructField("distance", FloatType(), True)
  ])


  recommendations_df = spark.createDataFrame(data=aggregated.items(), schema=recommendations_schema)

  return recommendations_df

if DEBUG:
  k = 40
  recommendations_df = aggregate_recommendations(recommendations)
  recommendations_df.show(truncate=False)
  first_playlist.show(truncate=False)
  print(recommendations_df.count(), first_playlist.count())

+------------------------------------+----------+
|track_uri                           |distance  |
+------------------------------------+----------+
|spotify:track:4Hj5yNHgu2dyrnzRQN7Gld|0.0       |
|spotify:track:1jNyxG5S2P9gztbfAnrq85|0.84      |
|spotify:track:34yAAS72vZUXvsQPHGZexE|0.85714287|
|spotify:track:7yQ2NWAIeDcIEauyVG5lck|0.884058  |
|spotify:track:0GS2K1xle0UpcAWa3u4BnG|0.88461536|
|spotify:track:366DLDHAnvMnHP5ECYt4N7|0.8901099 |
|spotify:track:7sdqtgsO9zxvKdrrVDpaNu|0.9027778 |
|spotify:track:45oGS3iHeT2cvkihSP7KzK|0.9041096 |
|spotify:track:2TLlFL6dpycifLqKtTE7UZ|0.91463417|
|spotify:track:4wlR30YY9mb1CsQkf9eoTP|0.9411765 |
|spotify:track:53mrVsi49rLHIaKBiSvElG|0.0       |
|spotify:track:2ekn2ttSfGqwhhate0LSR0|0.89898324|
|spotify:track:2771LMNxwf62FTAdpJMQfM|0.8064928 |
|spotify:track:7nZmah2llfvLDiUjm0kiyz|0.9093615 |
|spotify:track:4xBjsLUSjcx2h7PcRikpQp|0.93406594|
|spotify:track:3ojTJaonfkL96iIWa47SU3|0.93568146|
|spotify:track:7wGoVu4Dady5GV0Sv4UIsx|0.8632018 |


+---+---------------+------------------------------------+-------------------------------------+-----------------------------------------------------+------------------------------------+-----------+-----------------------------------------------------+
|pos|artist_name    |track_uri                           |artist_uri                           |track_name                                           |album_uri                           |duration_ms|album_name                                           |
+---+---------------+------------------------------------+-------------------------------------+-----------------------------------------------------+------------------------------------+-----------+-----------------------------------------------------+
|18 |Aminé          |spotify:track:4Hj5yNHgu2dyrnzRQN7Gld|spotify:artist:3Gm5F95VdRxW3mqCn8RPBJ|Yellow                                               |spotify:album:3lajefIuUk4SfzqVBSJy8p|180000     |Good For You                           

136 21


Now that we have a prediction, we can put the result in a pyspark dataframe in order to be used later

In [32]:
def remove_existing_tracks(playlist_tracks: DataFrame, recommendations_df: DataFrame) -> DataFrame:
  playlist_tracks = playlist_tracks.select("track_uri").cache()
  playlist_tracks_compatible = playlist_tracks.join(F.broadcast(recommendations_df), on="track_uri")
  playlist_tracks.unpersist()
  return recommendations_df.exceptAll(playlist_tracks_compatible)

if DEBUG:
  clean_df = remove_existing_tracks(first_playlist, recommendations_df)
  clean_df.show()

+--------------------+----------+
|           track_uri|  distance|
+--------------------+----------+
|spotify:track:7wG...| 0.8632018|
|spotify:track:34y...|0.85714287|
|spotify:track:1jN...|      0.84|
|spotify:track:4xB...|0.93406594|
|spotify:track:45o...| 0.9041096|
|spotify:track:7yQ...|  0.884058|
|spotify:track:4wl...| 0.9411765|
|spotify:track:0GS...|0.88461536|
|spotify:track:7sd...| 0.9027778|
|spotify:track:7nZ...| 0.9093615|
|spotify:track:366...| 0.8901099|
|spotify:track:2TL...|0.91463417|
|spotify:track:3oj...|0.93568146|
|spotify:track:2ek...|0.89898324|
|spotify:track:277...| 0.8064928|
|spotify:track:59J...|0.81729203|
|spotify:track:1sC...|0.86996084|
|spotify:track:3A7...| 0.8826916|
|spotify:track:7hC...| 0.9221902|
|spotify:track:1WI...|0.81096107|
+--------------------+----------+
only showing top 20 rows



Putting all togheter:

In [28]:
def item_based_recommendation(playlist: DataFrame,playlist_map: DataFrame, model: MinHashLSHModel, k=50):
  playlist_songs = playlist.select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(playlist_songs, playlist_map, model, 20, disable_pbar=True)
  recommendations_df = aggregate_recommendations(recommendations)
  recommendations_df = remove_existing_tracks(playlist_songs, recommendations_df)
  return recommendations_df.orderBy(F.col("distance").asc()).limit(k).cache()

if DEBUG:
  playlist = train_df.filter("pid == 2005")
  result = item_based_recommendation(playlist, playlist_map, model)
  result.show()

23/06/22 20:09:29 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.


+--------------------+----------+
|           track_uri|  distance|
+--------------------+----------+
|spotify:track:2t1...|0.64285713|
|spotify:track:2AO...| 0.6969697|
|spotify:track:2gh...|       0.7|
|spotify:track:5IF...|0.71900827|
|spotify:track:2ru...|0.77286583|
|spotify:track:5Qf...| 0.7910448|
|spotify:track:1Vu...| 0.8076923|
|spotify:track:6jL...|0.81005585|
|spotify:track:1ak...|0.81595093|
|spotify:track:4ej...| 0.8244275|
|spotify:track:2wm...|0.82828283|
|spotify:track:2BZ...|0.83035713|
|spotify:track:7qi...| 0.8330721|
|spotify:track:4I3...| 0.8359133|
|spotify:track:0b0...|0.83709276|
|spotify:track:54t...| 0.8402256|
|spotify:track:0Su...| 0.8404702|
|spotify:track:7nY...| 0.8422051|
|spotify:track:5Vw...| 0.8450494|
|spotify:track:72j...| 0.8450756|
+--------------------+----------+
only showing top 20 rows



# Performance Evaluation

In [29]:
def precision_at_k(recommendations, ground_truth, num_of_recommendations) -> float:
    """
    Calculates precision at k for the recommendations.
    """
    recommended_relevant_tracks = recommendations.join(ground_truth, "track_uri").cache()
    reccomended_relevant_tracks_count = recommended_relevant_tracks.count() #this can be top_n_results.join in order to be more performant
    recommended_relevant_tracks.unpersist()
    precision = reccomended_relevant_tracks_count / float(num_of_recommendations)

    return precision


import math
#TODO: make it more efficient somehow
def normalized_discounted_cumulative_gain(recommendations: DataFrame, ground_truth: DataFrame, num_of_recommendations: int) -> float:
  recommendations = recommendations.orderBy(F.col("distance").asc())
  recommendations_list = recommendations.collect()
  cumulative_gain = 0

  intersection = recommendations.join(ground_truth, "track_uri").count()
  if intersection == 0: return 0

  ideal_cumulative_gain = 1 + sum((1 / math.log(i, 2)) for i in range(2, 2+intersection))
  for index, row in enumerate(recommendations_list):
    i = index + 1
    is_rel = ground_truth.filter(F.col("track_uri").isin(row.track_uri)).count() > 0
    rel = 1 if is_rel else 0
    if i == 1:
      cumulative_gain += rel
    else:
      cumulative_gain += (rel / math.log(i, 2))
  return cumulative_gain / ideal_cumulative_gain

Executing performance evaluation on a random sample of the test set

In [31]:
def evaluate(pid: int, playlist_map) -> Tuple[DataFrame, float]:
    t1 = time.time()

    playlist_train = train_df.filter(f"pid == {pid}").cache()
    playlist_test = test_df.filter(f"pid == {pid}").cache()
    ground_truth = playlist_test.select(F.explode("tracks")).select("col.*").cache()
    num_of_recommendations = ground_truth.count()
    recommendations = item_based_recommendation(playlist_train, playlist_map, model, k=num_of_recommendations).cache()

    precision = precision_at_k(recommendations, ground_truth, num_of_recommendations)
    gain = normalized_discounted_cumulative_gain(recommendations, ground_truth, num_of_recommendations)

    t2 = time.time()
    print(f"Total time: {t2-t1}")

    playlist_train.unpersist()
    playlist_test.unpersist()
    ground_truth.unpersist()
    recommendations.unpersist()

    return playlist_train, playlist_test, ground_truth, recommendations, precision, gain


if DEBUG:
  train, test, gt, rec, prec, gain  = evaluate(1005, playlist_map)
  train.show(), test.show(), gt.show(), rec.show(truncate=False)
  print(f"Precision: {prec}, Gain: {gain}")

23/06/22 20:12:27 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.
23/06/22 20:12:27 WARN CacheManager: Asked to cache already cached data.


Total time: 114.25674796104431


23/06/22 20:12:28 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.
23/06/22 20:12:28 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.


+-------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|   name|collaborative| pid|modified_at|num_tracks|num_albums|num_followers|              tracks|num_edits|duration_ms|num_artists|
+-------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|college|        false|1005| 1504310400|        28|        26|            1|[{22, Kid Ink, sp...|        2|    5870710|         25|
+-------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+

+-------+-------------+----+-----------+----------+----------+-------------+--------------------+---------+-----------+-----------+
|   name|collaborative| pid|modified_at|num_tracks|num_albums|num_followers|              tracks|num_edits|duration_ms|num_artists|
+-------+-------------+----+-----------+----------+----------+-------------

23/06/22 20:12:30 WARN HintErrorLogger: A join hint (strategy=broadcast) is specified but it is not part of a join relation.


+------------------------------------+----------+
|track_uri                           |distance  |
+------------------------------------+----------+
|spotify:track:5tRmoZRoupvMYPV1oezFrH|0.69382393|
|spotify:track:0SGkqnVQo9KPytSri1H6cF|0.7495854 |
|spotify:track:7KOlJ92bu51cltsD9KU5I7|0.7688978 |
|spotify:track:1QgTBfRN5m81a5K01qens6|0.7697702 |
|spotify:track:5bgwqaRSS3M8WHWruHgSL5|0.79055595|
|spotify:track:0UioblV1x795s55Ur58c6c|0.79651016|
|spotify:track:1MJErNPIQUoEMu4YUUmecm|0.8027972 |
+------------------------------------+----------+

Precision: 0.0, Gain: 0


In [33]:
import json
EVALUATION_RESULTS_PATH = os.path.join(GDRIVE_DATA_DIR, "item_based_evaluation_results")
def perform_evaluation():
  # SAMPLING_FRACTION = 0.01
  SAMPLING_FRACTION = 0.0001
  sampled_playlists = train_df.sample(False, SAMPLING_FRACTION, seed=42).cache()

  transformed_playlist_map = model.transform(playlist_map).cache()
  results = []
  for row in tqdm(sampled_playlists.collect(), desc="Performing evaluation"):
      pid = row['pid']
      train, test, gt, rec, prec, gain = evaluate(pid,transformed_playlist_map)
      print((prec, gain))
      results.append((prec, gain))
  
  sampled_playlists.unpersist() 
  with open(EVALUATION_RESULTS_PATH, "w") as f:
    json.dump(results, f)
  return results

results = perform_evaluation()

Performing evaluation:   0%|          | 0/8 [00:00<?, ?it/s]

23/06/22 20:14:20 WARN CacheManager: Asked to cache already cached data.        


Total time: 70.87952995300293
(0.23076923076923078, 0.8403030283801005)


23/06/22 20:15:12 WARN CacheManager: Asked to cache already cached data.        


Total time: 47.532137870788574
(0.16666666666666666, 0.25)


...

Total time: 21.297004222869873
(0.0, 0)


23/06/22 20:22:54 WARN CacheManager: Asked to cache already cached data.        


Total time: 215.87304282188416
(0.16071428571428573, 0.7963065271236838)


23/06/22 20:23:38 WARN CacheManager: Asked to cache already cached data.        


Total time: 42.10801410675049
(0.1, 0.16666666666666666)


In [34]:
avg_prec = np.array(results).mean()
avg_prec, avg_gain = 0, 0
for prec, gain in results:
  avg_prec += prec
  avg_gain += gain 
tot = len(results)
avg_prec /= tot
avg_gain /= tot
avg_prec, avg_gain

(0.2040220196470197, 0.4610435126753924)

In [35]:
results

[(0.23076923076923078, 0.8403030283801005),
 (0.16666666666666666, 0.25),
 (0.045454545454545456, 0.7601875334318686),
 (0.42857142857142855, 0.37488434580081953),
 (0.5, 0.5),
 (0.0, 0),
 (0.16071428571428573, 0.7963065271236838),
 (0.1, 0.16666666666666666)]

# All against all tries

Since computing the k nearest neighbors is super slow, I can pre-compute them offline and store them. This will require like 100 years lol.

In [36]:
mh = MinHashLSH(inputCol="embedding", outputCol="hashes", numHashTables=5)
model = mh.fit(playlist_map)

In [ ]:
def compute_all_k_neighbors(playlist_map: DataFrame, model) -> DataFrame:
    result = []
    transformed_playlist_map = model.transform(playlist_map).cache()
    for index, row in enumerate(tqdm(playlist_map.collect(), desc="Computing k-neighbors")):
        k_neighs = model.approxNearestNeighbors(transformed_playlist_map, row.embedding, 10).select("track_uri", F.col("distCol").alias("similarity"))
        result.append((row.track_uri, k_neighs.collect()))

    k_neighs_schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("distCol", FloatType(), nullable=True)
    ])

    schema = StructType([
        StructField("track_uri", StringType(), nullable=True),
        StructField("k_neighs", ArrayType(k_neighs_schema), nullable=True)
    ])

    result_df = spark.createDataFrame(result, schema)
    transformed_playlist_map.unpersist()

    return result_df

result_df = compute_all_k_neighbors(playlist_map, model)

Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Read timed out
	at java.base/sun.nio.ch.NioSocketImpl.timedRead(NioSocketImpl.java:273)
	at java.base/sun.nio.ch.NioSocketImpl.implRead(NioSocketImpl.java:299)
	at java.base/sun.nio.ch.NioSocketImpl.read(NioSocketImpl.java:340)
	at java.base/sun.nio.ch.NioSocketImpl$1.read(NioSocketImpl.java:789)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1025)
	at java.base/java.net.Socket$SocketInputStream.read(Socket.java:1019)
	at java.base/java.io.DataInputStream.readInt(DataInputStream.java:381)
	at org.apache.spark.security.SocketAuthHelper.readUtf8(SocketAuthHelper.scala:103)
	at org.apache.spark.security.SocketAuthHelper.authClient(SocketAuthHelper.scala:57)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:67)


In [ ]:
K_NEIGHBOURS_PATH = os.path.join(GDRIVE_DATA_DIR,"saved_models", f"k_neighbours-{NUM_PLAYLISTS}.parquet")
result_df.write.parquet(K_NEIGHBOURS_PATH)

# Old stuff

In [ ]:
def transform_to_aggregate(df: DataFrame) -> DataFrame:
  """
  Transforms the dataframe in order to be compatible for the union with an aggregate df.
  """
  return df.withColumn("sum", F.col("similarity")).withColumnRenamed("similarity", "squared_sum")

def merge_aggregate_df(aggregate_df: DataFrame, other_df: DataFrame) -> DataFrame:
  """
  Sums an aggregate df (track_uri, embedding, hashes, squared_sum, sum)
  with a simple df (track_uri, embedding, hashes, distCol).
  """
  other_df = transform_to_aggregate(other_df)
  merged_df = aggregate_df.unionAll(other_df)
  return merged_df.groupBy("track_uri", "embedding").agg(
    F.sum(F.pow("squared_sum", 2)).alias("squared_sum"),
    F.sum("sum").alias("sum"))

In [ ]:
def create_similarity_df(input_vector: SparseVector, playlist_map: DataFrame, similarity_function: Callable) -> DataFrame:
  """
  Returns the similarity df for a single song
  """
  
  @F.udf(returnType=FloatType())
  def compute_similarity(vector1):
    return similarity_function(vector1, input_vector)

  result_df = playlist_map.withColumn("similarity", compute_similarity(playlist_map.embedding))
  
  return result_df

def get_top_k_results(track_uri: str, similarity_df: DataFrame, k: int = 20) -> DataFrame:
  return similarity_df.filter((F.col("similarity") > 0) & (F.col("track_uri") != track_uri)).orderBy(F.col("similarity").desc()).limit(k)

if DEBUG:
  first_song_vector = playlist_map.select("embedding").limit(1).first()[0]
  first_track_uri = playlist_map.select("track_uri").limit(1).first()[0]
  similarity_df = create_similarity_df(first_song_vector, playlist_map, jaccard_similarity)
  top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
  top_k_results.show()

In [ ]:
def extract_similar_songs(playlist_tracks: DataFrame, k=10) -> DataFrame:
  aggregate_df = None
  tracks_embedding = playlist_tracks.join(playlist_map, "track_uri")
  for row in tqdm(tracks_embedding.collect(), desc='Extracting k-neighbors'):
    k_neigh = get_top_k_results(
        track_uri=row.track_uri,
        similarity_df= create_similarity_df(input_vector=row.embedding,
                                            playlist_map=playlist_map,
                                            similarity_function=jaccard_similarity),
        k = 10)
    
    k_neigh.show()
    if aggregate_df == None:
      aggregate_df = transform_to_aggregate(k_neigh).cache()
    else:
      aggregate_df = merge_aggregate_df(aggregate_df, k_neigh).cache()
    
    aggregate_df.show()
  return aggregate_df

if DEBUG:
  first_playlist = train_df.limit(1).select(F.explode("tracks")).select("col.*")
  recommendations = extract_similar_songs(first_playlist, model)

In [ ]:
recommendations.show()

In [ ]:
# def extract_similar_songs(playlist: DataFrame, playlist_map: DataFrame) -> Dict[str, float]:
#   """
#   For each track in the playlist, it extracts the top-k most similar results.
#   """
#   weighted_dict = {}
  
#   # F.udf(returnType=IntegerType())
#   def update_weighted_dict(track_uri: str, similarity: float):
#     nonlocal weighted_dict
#     if track_uri not in weighted_dict:
#       weighted_dict[track_uri] = (similarity, 1)
#     else:
#       curr_similarity = weighted_dict[track_uri][0]
#       count = weighted_dict[track_uri][1]
#       weighted_dict[track_uri] = (curr_similarity + similarity, count+1)
  
#   tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
#   tracks_mapped = tracks.join(playlist_map, "track_uri")

#   for track_row in tqdm(tracks_mapped.collect(), desc=f"Analyzing tracks"):
#     similarity_df = create_similarity_df(track_row.embedding, playlist_map, jaccard_similarity)
#     top_k_results = get_top_k_results(first_track_uri, similarity_df).cache()
#     top_k_results.foreach(lambda row: update_weighted_dict(row.track_uri, row.similarity))
#     top_k_results.unpersist()
  
#   return weighted_dict

# if DEBUG:
#   first_playlist = train_df.limit(1).cache()
#   weighted_dict = extract_similar_songs(first_playlist, playlist_map)

In [ ]:
def create_similarity_df(playlist: DataFrame, playlist_map: DataFrame, similarityFunction: Callable):
  # Get the current playlist's tracks
  playlist_tracks = playlist.select(F.explode("tracks.track_uri")).withColumnRenamed("col", "track_uri")
  playlist_tracks.show()
  # Let's extract the rating vector for each single track
  playlist_tracks_mapped = playlist_tracks.join(playlist_map, "track_uri")
  playlist_tracks_mapped.show()


  @F.udf(returnType=FloatType())
  def compute_similarity(input_vector: SparseVector, other_vector: SparseVector):
    return jaccard_similarity(input_vector, other_vector)

  # for row in playlist_tracks_mapped.collect():
  similarity_df = playlist_map.withColumn("similarity", compute_similarity(playlist_tracks_mapped.limit(2).embedding, playlist_map.embedding))
  # break
  
  similarity_df.show()
  return

first_playlsit = train_df.limit(1).cache()
create_similarity_df(first_playlsit, playlist_map, jaccard_similarity)

In [ ]:
first_playlsit.show()